

# Algoritmo Differential Evolution (DE)

## Introducción

La Evolución Diferencial (Differential Evolution, DE) es un algoritmo de optimización global basado en población, perteneciente a la familia de los algoritmos evolutivos. Fue introducido por Storn y Price en 1997. Al igual que otros métodos evolutivos (como los Algoritmos Genéticos), DE utiliza mecanismos inspirados en la biología, como la mutación, el cruce (crossover) y la selección, para encontrar el mínimo (o máximo) de una función objetivo.

### Características Clave de DE:

  * **Simple y Robusto:** Es conceptualmente simple y requiere pocos parámetros de control.
  * **Eficaz para Optimización Global:** Es muy efectivo en problemas de optimización continua no lineal con múltiples mínimos locales.
  * **Mecanismo de Mutación:** La principal diferencia con otros algoritmos es su innovador método de mutación basado en la diferencia de vectores de la población.

-----

## Fundamentos Teóricos del DE Clásico

El algoritmo DE trabaja con una población de $NP$ (Número de Población) vectores de $D$ (Dimensión del problema) parámetros (variables de decisión). Sea $\mathbf{x}_{i, G}$ el $i$-ésimo vector de la población en la generación (iteración) $G$.

El proceso se repite por un número máximo de generaciones ($G_{max}$) o hasta que se cumpla un criterio de convergencia, y consta de 4 pasos principales:

1.  Inicialización
2.  Mutación
3.  Cruce (Recombinación)
4.  Selección

### 1\. Inicialización

La población inicial ($G=0$) se genera aleatoriamente de forma uniforme dentro de los límites del espacio de búsqueda, $[\mathbf{x}_{\min}, \mathbf{x}_{\max}]$.

$$\mathbf{x}_{i, 0} = \mathbf{x}_{\min} + \text{rand}(0, 1) \cdot (\mathbf{x}_{\max} - \mathbf{x}_{\min})$$

Donde $\mathbf{x}_{i, 0}$ es el $i$-ésimo individuo, y $\text{rand}(0, 1)$ es un número aleatorio uniformemente distribuido entre 0 y 1.

### 2\. Mutación

Para cada vector objetivo $\mathbf{x}_{i, G}$ (el *mutante* potencial), se genera un **vector mutante** $\mathbf{v}_{i, G+1}$ utilizando la información de **tres vectores distintos** de la población actual, $\mathbf{x}_{r_1, G}$, $\mathbf{x}_{r_2, G}$, y $\mathbf{x}_{r_3, G}$, donde $r_1, r_2, r_3 \in \{1, 2, \ldots, NP\}$ son índices aleatorios y distintos entre sí, y distintos del índice objetivo $i$.

La estrategia de mutación más común (DE/rand/1) es:

$$\mathbf{v}_{i, G+1} = \mathbf{x}_{r_1, G} + F \cdot (\mathbf{x}_{r_2, G} - \mathbf{x}_{r_3, G})$$

Donde:

  * $\mathbf{v}_{i, G+1}$ es el **vector mutante**.
  * $F \in [0, 2]$ es el **Factor de Escalamiento** (o *Factor de Mutación*), un parámetro de control que determina la magnitud de la perturbación.

### 3\. Cruce (Recombinación)

El vector objetivo $\mathbf{x}_{i, G}$ y el vector donante $\mathbf{v}_{i, G+1}$ se combinan para crear un **vector de prueba** (o *trial*) $\mathbf{u}_{i, G+1}$.

Cada componente $j$ del vector de prueba se toma del vector donante $\mathbf{v}_{i, G+1}$ con una probabilidad $CR$ (Tasa de Cruce), o del vector objetivo $\mathbf{x}_{i, G}$ con probabilidad $1 - CR$.

$$u_{j, i, G+1} = \begin{cases} v_{j, i, G+1} & \text{si } \text{rand}_j(0, 1) \le CR \text{ o } j = j_{\text{rand}} \\ x_{j, i, G} & \text{en otro caso} \end{cases}$$

Donde:

  * $CR \in [0, 1]$ es la **Tasa de Cruce**, un parámetro de control.
  * $\text{rand}_j(0, 1)$ es un número aleatorio uniformemente distribuido entre 0 y 1 para cada componente $j$.
  * $j_{\text{rand}}$ es un índice aleatorio elegido para asegurar que al menos una componente del vector donante pase al vector de prueba, garantizando una nueva solución.

### 4\. Selección

El vector de prueba $\mathbf{u}_{i, G+1}$ compite con el vector objetivo original $\mathbf{x}_{i, G}$. El que tenga un mejor valor de la función objetivo (menor costo en minimización) sobrevive y forma parte de la población de la siguiente generación ($G+1$).

$$\mathbf{x}_{i, G+1} = \begin{cases} \mathbf{u}_{i, G+1} & \text{si } f(\mathbf{u}_{i, G+1}) \le f(\mathbf{x}_{i, G}) \\ \mathbf{x}_{i, G} & \text{en otro caso} \end{cases}$$

En el algoritmo DE, el operador de mutación ayuda a que las partículas exploren el espacio de búsqueda con el objetivo de encontrar zonas potenciales en las que pudiera estár la solución óptima, mientras que el operador de selección ayuda a explotar las zonas potenciales, refinando la solución y mejorándola en cada generación.
-----

## Implementación Paso a Paso con Python

Usaremos Python con `numpy` para la implementación y `matplotlib` para la visualización.

### Parámetros del Problema y del Algoritmo

| Parámetro | Símbolo | Valor Sugerido | Descripción |
| :--- | :--- | :--- | :--- |
| Dimensión | $D$ | 2 | Número de variables de decisión. |
| Límites | $[\mathbf{x}_{\min}, \mathbf{x}_{\max}]$ | $[-3, 3]$ | Rango de búsqueda para cada variable. |
| Población | $NP$ | 10 | Número de vectores en la población. |
| Generaciones | $G_{max}$ | 50 | Número máximo de iteraciones. |
| Factor de Escalamiento | $F$ | 0.8 | Controla la magnitud de la mutación. |
| Tasa de Cruce | $CR$ | 0.9 | Controla qué tan mezclados están los vectores. |

### Función de Costo de Ejemplo: Peaks

Utilizaremos la función `peaks` de MATLAB, una función de prueba multimodales clásica en dos dimensiones.

$$f(x, y) = 3(1-x)^2e^{-x^2 - (y+1)^2} - 10(\frac{x}{5} - x^3 - y^5)e^{-x^2-y^2} - \frac{1}{3}e^{-(x+1)^2 - y^2}$$

El mínimo global de esta función en el rango $[-3, 3]$ es aproximadamente $-6.5511$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# ----------------------------------------------------
# 0. Definición de la Función Objetivo (Peaks)
# ----------------------------------------------------

def peaks(x):
    """
    Implementación de la función Peaks (2D)
    x es un array de numpy [x1, x2]
    """
    x1 = x[0]
    x2 = x[1]

    # Término 1
    t1 = 3 * (1 - x1)**2 * np.exp(-(x1**2) - (x2 + 1)**2)

    # Término 2
    t2 = -10 * (x1/5 - x1**3 - x2**5) * np.exp(-(x1**2) - x2**2)

    # Término 3
    t3 = -1/3 * np.exp(-(x1 + 1)**2 - x2**2)

    return t1 + t2 + t3

# ----------------------------------------------------
# 1. Configuración e Inicialización
# ----------------------------------------------------

# Parámetros del Algoritmo DE
NP = 10       # Tamaño de la población
D = 2         # Dimensión del problema
G_max = 50    # Número máximo de generaciones
F = 0.8       # Factor de Escalamiento (Mutación)
CR = 0.9      # Tasa de Cruce

# Límites del Espacio de Búsqueda
bounds = np.array([[-3.0, 3.0], [-3.0, 3.0]])

# Inicialización de la población (matriz NP x D)
P = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(NP, D))
# Evaluación inicial
fitness = np.array([peaks(vec) for vec in P])

# Almacenamiento de resultados para visualización
history = [P.copy()]
best_fitness_history = [np.min(fitness)]

# ----------------------------------------------------
# 2. Ciclo Principal del DE
# ----------------------------------------------------

for G in range(G_max):
    # Crear la nueva población para la generación G+1
    new_P = np.zeros_like(P)
    new_fitness = np.zeros_like(fitness)

    for i in range(NP):
        # --------------------------------------------
        # 2a. Mutación (DE/rand/1)
        # --------------------------------------------

        # Seleccionar 3 índices distintos: r1, r2, r3 != i
        indices = [j for j in range(NP) if j != i]
        r1, r2, r3 = P[np.random.choice(indices, 3, replace=False)]

        # Vector mutante
        v = r1 + F * (r2 - r3)

        # --------------------------------------------
        # 2b. Cruce (Recombinación)
        # --------------------------------------------

        u = np.zeros(D) # Vector de prueba
        j_rand = np.random.randint(0, D) # Asegurar al menos un componente

        for j in range(D):
            if np.random.rand() <= CR or j == j_rand:
                u[j] = v[j]
            else:
                u[j] = P[i, j] # Tomar del vector objetivo

        # Asegurar que el vector de prueba permanezca dentro de los límites
        u = np.clip(u, bounds[:, 0], bounds[:, 1])

        # --------------------------------------------
        # 2c. Selección
        # --------------------------------------------

        f_u = peaks(u) # Evaluación del vector de prueba

        if f_u <= fitness[i]: # Minimización: Si es mejor o igual, reemplaza
            new_P[i] = u
            new_fitness[i] = f_u
        else:
            new_P[i] = P[i]
            new_fitness[i] = fitness[i]

    # Actualizar la población y el fitness
    P = new_P
    fitness = new_fitness

    # Guardar estado para la visualización
    history.append(P.copy())
    best_fitness_history.append(np.min(fitness))

# Resultados Finales
best_index = np.argmin(fitness)
best_solution = P[best_index]
best_cost = fitness[best_index]

print(f"\n Optimización Completada después de {G_max} generaciones.")
print(f"Mejor Solución Encontrada: x = {best_solution[0]:.4f}, y = {best_solution[1]:.4f}")
print(f"Costo Mínimo (Peaks): {best_cost:.4f}")
print(f"Óptimo Global Teórico (Peaks): -6.5511")

## Visualización de Resultados

### 1\. Evolución del Mejor Costo

Esta gráfica muestra cómo el valor de la función objetivo del mejor individuo en la población disminuye (converge) a lo largo de las generaciones.

In [ ]:
# Gráfica de Convergencia
plt.figure(figsize=(10, 6))
plt.plot(best_fitness_history, label='Mejor Costo (DE)', color='darkorange')
plt.hlines(-6.5511, 0, G_max, colors='r', linestyles='--', label='Óptimo Teórico')
plt.title('Evolución del Mejor Costo a lo Largo de las Generaciones')
plt.xlabel('Generación')
plt.ylabel('Valor de la Función Peaks')
plt.legend()
plt.grid(True)
plt.show()

### 2\. Animación de la evolución de Partículas

Esta animación muestra el movimiento de toda la población de vectores sobre la superficie de la función `peaks`. Se puede apreciar cómo las partículas se agrupan en torno al mínimo global con el paso de las iteraciones.

In [ ]:
# ----------------------------------------------------
# Visualización de la Superficie Peaks
# ----------------------------------------------------
x = np.linspace(bounds[0, 0], bounds[0, 1], 100)
y = np.linspace(bounds[1, 0], bounds[1, 1], 100)
X, Y = np.meshgrid(x, y)
Z = np.array([[peaks(np.array([xi, yi])) for xi in x] for yi in y])

fig, ax = plt.subplots(figsize=(5, 5))
# Trazar el mapa de contorno de la función Peaks
ax.contourf(X, Y, Z, levels=50, cmap='viridis')
ax.set_title('Búsqueda del Algoritmo DE sobre la Función Peaks')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.set_xlim(bounds[0])
ax.set_ylim(bounds[1])

# Crear el scatter plot de la población
scatter = ax.scatter([], [], c='r', marker='o', s=30)
best_point, = ax.plot([], [], 'w*', markersize=12, label='Mejor Solución')

# Función de inicialización de la animación
def init_anim():
    scatter.set_offsets(np.empty((0, 2))) # Fixed: Pass an empty 2D array
    best_point.set_data([], [])
    return scatter, best_point,

# Función de actualización para cada frame
def animate(i):
    # Coordenadas de la población en la generación i
    current_P = history[i]

    # Actualizar la posición de todos los puntos
    scatter.set_offsets(current_P)

    # Encontrar y actualizar el mejor punto de la población actual
    current_fitness = np.array([peaks(vec) for vec in current_P])
    best_idx = np.argmin(current_fitness)
    best_sol = current_P[best_idx]

    # Fix: Wrap scalar coordinates in lists
    best_point.set_data([best_sol[0]], [best_sol[1]])

    ax.set_title(f'Generación {i+1}/{G_max}')
    return scatter, best_point,

# Crear la animación
anim = animation.FuncAnimation(
    fig, animate, init_func=init_anim,
    frames=len(history), interval=200, blit=True
)

# Mostrar la animación
# Nota: Para ejecutar esto en un notebook, descomenta la línea siguiente:
HTML(anim.to_jshtml())
# plt.show()

-----

## Conclusiones

El DE es una poderosa herramienta de optimización global. Como se observa en la gráfica de convergencia, el algoritmo reduce rápidamente el valor de la función objetivo, acercándose al mínimo global teórico.

La animación muestra el mecanismo del DE:

1.  En las primeras generaciones, los puntos de la población están dispersos, explorando el espacio de búsqueda.
2.  A medida que avanza el algoritmo, la población comienza a concentrarse en las regiones de bajo costo, especialmente en el mínimo global, demostrando el proceso de explotación (refinamiento) de la búsqueda.
